In [2]:
import torch 
import torch.autograd as autograd
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim 

device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(1)

data = [('me gusta comer en la cafeteria'.split(), 'SPANISH'),
        ('Give it to me'.split(), 'ENGLISH'),
        ('No creo que sea. una buena idea'.split(), 'SPANISH'),
        ('No it is not a good idea to get lost at sea'.split(), 'ENGLISH')]

test_data = [('Yo creo que si'.split(), 'SPANSIH'),
             ('it is lost on me'.split(),'ENGLISH')]

word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word]  = len(word_to_ix)
            
label_to_ix = {"SPANISH": 0, "ENGLISH": 1}
print('word_to_ix:\n',word_to_ix)
print('label_to_ix:\n',label_to_ix)

def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] = 1
    return vec.view(1,-1)
        
def make_target(label, label_to_ix):    
    return torch.LongTensor([label_to_ix[label]])
    
class BoWClassfier(nn.Module):
    
    def __init__(self, input_size, output_size):
        super(BoWClassfier, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.fc = nn.Linear(self.input_size, self.output_size)

    def forward(self, inputs):
        outs = F.log_softmax(self.fc(inputs), dim=1)
        return outs 
    
input_size = len(word_to_ix) # vocab_size
output_size = 2 # num_label

model = BoWClassfier(input_size, output_size).to(device) 
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(100):
    for instance, label in data:
        model.zero_grad() 
        bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix)).to(device)
        target = autograd.Variable(make_target(label, label_to_ix)).to(device)
        log_probs = model(bow_vec)
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec.to(device))
    print(log_probs)

print(next(model.parameters())[:,word_to_ix['creo']])    

word_to_ix:
 {'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea.': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'sea': 23, 'Yo': 24, 'si': 25, 'on': 26}
label_to_ix:
 {'SPANISH': 0, 'ENGLISH': 1}
tensor([[-0.1512, -1.9637]], device='cuda:0', grad_fn=<LogSoftmaxBackward>)
tensor([[-2.3918, -0.0959]], device='cuda:0', grad_fn=<LogSoftmaxBackward>)
tensor([ 0.4763, -0.4550], device='cuda:0', grad_fn=<SelectBackward>)


In [ ]:
class BoWClassfier(nn.Module):
    
    def __init__(self, input_size, output_size):
        super(BoWClassfier, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.fc = nn.Linear(self.input_size, self.output_size)

    def forward(self, inputs):
        outs = F.log_softmax(self.fc(inputs), dim=1)
        
        return outs 
    
input_size = len(word_to_ix)
output_size = 2

model = BoWClassfier(input_size, output_size).to(device) 
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(100):
    for instance, label in data:
        model.zero_grad() 
        bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix)).to(device)
        target = autograd.Variable(make_target(label, label_to_ix)).to(device)
        log_probs = model(bow_vec)
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

In [4]:
y = torch.rand(5, 3)
print(y)

tensor([[0.3828, 0.7442, 0.5285],
        [0.6642, 0.6099, 0.6818],
        [0.7479, 0.0369, 0.7517],
        [0.1484, 0.1227, 0.5304],
        [0.4148, 0.7937, 0.2104]])


In [5]:
import ray
ray.init()

2020-12-23 18:20:42,684	INFO services.py:1169 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.1.161',
 'raylet_ip_address': '192.168.1.161',
 'redis_address': '192.168.1.161:6379',
 'object_store_address': '/tmp/ray/session_2020-12-23_18-20-42_126616_7535/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-12-23_18-20-42_126616_7535/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2020-12-23_18-20-42_126616_7535',
 'metrics_export_port': 50675,
 'node_id': 'e637bd94865b6fd5a6df93b625f68895c17cea41b4121546106507bc'}

In [24]:
import time
import ray
ray.shutdown()
ray.init()

@ray.remote
def A():
    time.sleep(10)
    print("print A run over")

@ray.remote
def B():
    time.sleep(20)
    print("print B run over")

task_A = A.remote()
task_B = B.remote()
task_list = [task_A,task_B]
print ("Start : %s" % time.ctime())
get_result = ray.get(task_list)
print ("End : %s" % time.ctime())

2020-12-24 09:26:50,996	INFO services.py:1169 -- View the Ray dashboard at http://127.0.0.1:8265


Start : Thu Dec 24 09:26:51 2020


(pid=raylet) E1224 09:26:51.501226182    8714 http_proxy.cc:78]           'https' scheme not supported in proxy URI
(pid=raylet) E1224 09:26:51.501226182    8714 http_proxy.cc:78]           'https' scheme not supported in proxy URI
(pid=raylet) E1224 09:26:51.501226182    8714 http_proxy.cc:78]           'https' scheme not supported in proxy URI
(pid=raylet) E1224 09:26:51.501226182    8714 http_proxy.cc:78]           'https' scheme not supported in proxy URI
(pid=raylet) E1224 09:26:51.501226182    8714 http_proxy.cc:78]           'https' scheme not supported in proxy URI
(pid=raylet) E1224 09:26:51.501226182    8714 http_proxy.cc:78]           'https' scheme not supported in proxy URI
(pid=raylet) E1224 09:26:51.501226182    8714 http_proxy.cc:78]           'https' scheme not supported in proxy URI
(pid=raylet) E1224 09:26:51.501226182    8714 http_proxy.cc:78]           'https' scheme not supported in proxy URI
(pid=raylet) E1224 09:26:51.501226182    8714 http_proxy.cc:78]         

(pid=8735) print A run over
(pid=8735) print A run over
(pid=8735) print A run over
(pid=8735) print A run over
(pid=8735) print A run over
(pid=8735) print A run over
(pid=8735) print A run over
(pid=8735) print A run over
(pid=8735) print A run over
(pid=8735) print A run over
End : Thu Dec 24 09:27:11 2020
(pid=8724) print B run over
(pid=8724) print B run over
(pid=8724) print B run over
(pid=8724) print B run over
(pid=8724) print B run over
(pid=8724) print B run over
(pid=8724) print B run over
(pid=8724) print B run over
(pid=8724) print B run over
(pid=8724) print B run over


In [1]:
import time 
import ray
#关闭ray，不关闭直接初始化ray可能会报错
ray.shutdown()
#初始化ray
ray.init()

@ray.remote(num_returns=2)
def ParallelFunction(num):
    time.sleep(5)
    print("the input number:",num)
    return num - 1, 5

r1,r2 = ParallelFunction.remote(10)
r3,r4 = ParallelFunction.remote(9)

task_list = [r1,r2,r3,r4]

r1,r2,r3,r4 = ray.get(task_list)

print("task1_return:",r1,r2)
print("task2_return:",r3,r4)

2021-01-27 13:00:56,137	INFO services.py:1169 -- View the Ray dashboard at http://127.0.0.1:8266


task1_return: 9 5
task2_return: 8 5
(pid=1334) the input number: 9
(pid=1339) the input number: 10
